# Twitter Sentiment Analysis

Scripping Twitter menggunakan library **SNScrape** [dokumentasi](https://github.com/JustAnotherArchivist/snscrape)

## 1. Initiation
Pada tahap inisiasi, akan dilakukan install library SNScrape, import library-library yang dibutuhkan, *mount* script ini ke drive, dan konfigurasi pandas

Install library SNScrape dapat dilakukan dengan script berikut:

In [1]:
!pip install snscrape
!pip install daterangeparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 KB 1.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Library yang digunakan adalah sntwitter dari snscrape. pandas dan numpy untuk mempermudah proses wrangling. dan library json, ast, re untuk mempermudah parsing data atau extraksi data

In [2]:
from dateutil import parser
from daterangeparser import parse
from datetime import datetime, timedelta
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
import json
import ast
import re
import os

Mount drive dapat dilakukan menggunakan script berikut untuk dapat mempermudah upload dan download file yang diinginkan

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Konfigurasi pandas untuk mempercantik dan bisa menampilkan kolom dan row dengan jumlah lebih banyak

In [4]:
# Pandas config
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
def parseStartEndDate(datestring):
  datestring = datestring.replace('.', '').strip()
  try:
    date_format = parse(datestring)[0]
  except:
    if datestring == '-':
      date_format = None
    else:
      date_format = datetime.strptime(datestring, "%d-%b-%y")
  return date_format

FILENAME = '/content/drive/MyDrive/Freelance/twitter_sentiment_analysis/data_new/Aksi_Korporasi_IPO_Tahun_2011_2022.csv'
DESTFILE = '/content/drive/MyDrive/Freelance/twitter_sentiment_analysis/data_new/output/'
df_kosongan = pd.read_csv(DESTFILE+"kosongan.csv").drop(columns = ['Unnamed: 0'])
df_fileipo = pd.read_csv(FILENAME, sep=';')
df_fileipo['tanggal_penawaran_awal'] = df_fileipo['Tanggal Penawaran Awal'].apply(parseStartEndDate)
df_fileipo['tanggal_ipo'] = pd.to_datetime(df_fileipo['Tanggal Pencatatan IPO'], format="%d-%b-%y")
df_fileipo['diff_days'] = (df_fileipo['tanggal_ipo'] - df_fileipo['tanggal_penawaran_awal']) / np.timedelta64(1, 'D')
df_fileipo['28_days_date'] = df_fileipo['tanggal_ipo'] -  pd.to_timedelta(28, unit='d')
df_fileipo['tgl_awal_coalesced'] = df_fileipo['tanggal_penawaran_awal'].combine_first(df_fileipo['28_days_date'])
df_fileipo['tgl_poin_1'] = df_fileipo['tgl_awal_coalesced'] -  pd.to_timedelta(77, unit='d')
df_fileipo['tanggal_ipo_plus_1'] = df_fileipo['tanggal_ipo'] +  pd.to_timedelta(1, unit='d')

df_fileipo['tanggal_penawaran_umum'] = df_fileipo['Masa Penawaran Umum'].apply(parseStartEndDate)
df_fileipo['pblcoffr'] = df_fileipo['tanggal_ipo'] -  pd.to_timedelta(8, unit='d')

df_fileipo['tanggal_penawaran_umum_coalesce'] = df_fileipo['tanggal_penawaran_umum'].combine_first(df_fileipo['pblcoffr'])

# df_fileipo = df_fileipo[df_fileipo['tanggal_ipo'].dt.year != 2022]
df_fileipo.to_csv('/content/drive/MyDrive/Freelance/twitter_sentiment_analysis/data_new/Aksi_Korporasi_IPO_Tahun_fix.csv')
df_fileipo.head()

,No.,Kode,Nama Emiten,Sektor,Tanggal Pencatatan IPO,Tanggal Penawaran Awal,Harga Penawaran Awal,Harga IPO,Masa Penawaran Umum,nama,kode,tanggal_penawaran_awal,tanggal_ipo,diff_days,28_days_date,tgl_awal_coalesced,tgl_poin_1,tanggal_ipo_plus_1,tanggal_penawaran_umum,pblcoffr,tanggal_penawaran_umum_coalesce
0,1,EMDE,Megapolitan Developments Tbk.,Properties & Real Estate,12-Jan-11,17 - 22 Dec 2010,150 - 250,250,4 - 6 Jan 2011,Megapolitan Developments,EMDE,2010-12-17,2011-01-12,26.0,2010-12-15,2010-12-17,2010-10-01,2011-01-13,2011-01-04,2011-01-04,2011-01-04
1,2,MBTO,Martina Berto Tbk.,Consumer Non-Cyclicals,13-Jan-11,-,650 - 850,740,3 - 7 Jan 2011,Martina Berto,MBTO,NaT,2011-01-13,NaN,2010-12-16,2010-12-16,2010-09-30,2011-01-14,2011-01-03,2011-01-05,2011-01-03
2,3,GIAA,Garuda Indonesia (Persero) Tbk.,Transportation & Logistic,11-Feb-11,-,-,750,2-8 Feb 2011,PT Garuda Indonesia,GIAA,NaT,2011-02-11,NaN,2011-01-14,2011-01-14,2010-10-29,2011-02-12,2011-02-02,2011-02-03,2011-02-02
3,4,MBSS,Mitrabahtera Segara Sejati Tbk.,Energy,06-Apr-11,-,1500-1900,1.600,29 - 31 Mar 2011,Mitrabahtera Segara Sejati,MBSS,NaT,2011-04-06,NaN,2011-03-09,2011-03-09,2010-12-22,2011-04-07,2011-03-29,2011-03-29,2011-03-29
4,5,SRAJ,Sejahteraraya Anugrahjaya Tbk.,Healthcare,11-Apr-11,-,110-130,120,4-6 Apr 2011,Sejahteraraya Anugrahjaya,SRAJ,NaT,2011-04-11,NaN,2011-03-14,2011-03-14,2010-12-27,2011-04-12,2011-04-04,2011-04-03,2011-04-04


In [6]:
FILENAME = '/content/drive/MyDrive/Freelance/twitter_sentiment_analysis/data_new/Aksi_Korporasi_IPO_Tahun_fix.csv'
df_fileipo = pd.read_csv(FILENAME).drop(columns = ['Unnamed: 0'])
df_fileipo[df_fileipo['kode'] == 'TRUE'].head()

,No.,Kode,Nama Emiten,Sektor,Tanggal Pencatatan IPO,Tanggal Penawaran Awal,Harga Penawaran Awal,Harga IPO,Masa Penawaran Umum,nama,kode,tanggal_penawaran_awal,tanggal_ipo,diff_days,28_days_date,tgl_awal_coalesced,tgl_poin_1,tanggal_ipo_plus_1,tanggal_penawaran_umum,pblcoffr,tanggal_penawaran_umum_coalesce
354,355,TRUE,PT Triniti Dinamik Tbk.,Properties & Real Estate,10-Jun-21,18 - 21 May 2021,100,100,3- 4 Jun 2021,Triniti Dinamik,TRUE,2021-05-18,2021-06-10,23.0,2021-05-13,2021-05-18,2021-03-02,2021-06-11,2021-06-03,2021-06-02,2021-06-03


## 2. Define Function
Dalam proses get twitter terdiri dari get data dengan kode saham "$kode" dan get data menggunakan 

In [ ]:
def get_tweets_union(kode, nama, since, until):
  attributes_container = []

  if kode in ['SIDO', 'PSSI', 'BOLT', 'MARK', 'ANDI', 'NASA', 'TOPS']:
    tweets_dfkd = df_kosongan
  else:
    # Using TwitterSearchScraper to scrape data and append tweets to list
    query = '{} since:{} until:{} lang:in'.format('$'+kode,since, until)
    enum = enumerate(sntwitter.TwitterSearchScraper(query).get_items())
    for i,tweet in enum:
      if i >= 50000:
        print('break')
        break
      attributes_container.append(json.loads(tweet.json()))    
    # Creating a dataframe to load the list
    tweets_dfkd = pd.DataFrame(attributes_container)

  attributes_container = []
  # Using TwitterSearchScraper to scrape data and append tweets to list
  query = '{} since:{} until:{} lang:in'.format(nama, since, until)
  enum = enumerate(sntwitter.TwitterSearchScraper(query).get_items())
  for i,tweet in enum:
    if i >= 50000:
      print('break')
      break
    attributes_container.append(json.loads(tweet.json()))
  # Creating a dataframe to load the list
  tweets_dfnm = pd.DataFrame(attributes_container)

  if kode in ['VIVA', 'SATU', 'REAL', 'NASA', 
              'BALI', 'PSSI', 'BANK', 'BUKA',
              'CITY', 'UANG', 'FILM']:
    tweets_dfshm = df_kosongan
  else:
    attributes_container = []
    # Using TwitterSearchScraper to scrape data and append tweets to list
    query = 'saham {} since:{} until:{} lang:in'.format(kode, since, until)
    enum = enumerate(sntwitter.TwitterSearchScraper(query).get_items())
    for i,tweet in enum:
      if i >= 50000:
        print('break')
        break
      attributes_container.append(json.loads(tweet.json()))
    # Creating a dataframe to load the list
    tweets_dfshm = pd.DataFrame(attributes_container)

  df_return = pd.concat([tweets_dfkd, tweets_dfnm, tweets_dfshm])
  df_return['id'] = df_return['id'].astype(str)
  df_return['conversationId'] = df_return['conversationId'].astype(str)
  df_return['inReplyToTweetId'] = df_return['inReplyToTweetId'].fillna(0).astype(np.int64).astype(str)
  return df_return

## 3.  Testing Functions

In [ ]:
# Test get GOTO
kode = 'GOTO'
nama = 'GoTo Gojek Tokopedia'
since = '2022-04-11'
until = '2022-04-12'
df_goto = get_tweets_union(kode, nama, since, until)
len(df_goto)
# df_goto['Date Created'] = df_goto['Date Created'].dt.tz_localize(None)
# .to_excel(DESTFILE+kode+".xlsx")

596

In [ ]:
df_goto = pd.read_excel(DESTFILE+kode+".xlsx")

In [ ]:
df_goto[df_goto.index == 6]['renderedContent'].to_list()

['Shm! Dapet saham gratis dari $GOTO bagusnya bikin sekuritas di mana ya?🙏 https://t.co/ZmooCcQteM']

In [ ]:
df_goto[df_goto.index == 6]['renderedContent'].to_list()

['Shm! Dapet saham gratis dari $GOTO bagusnya bikin sekuritas di mana ya?🙏 https://t.co/ZmooCcQteM',
 'PT GoTo Gojek Tokopedia Tbk mampu menghimpun dana hingga Rp 13,7 triliun dan memiliki kapitalisasi pasar mencapai Rp 400 triliun. #BEI m.jpnn.com/news/bei-goto-…',
 'Saham GOTO hari pertama naik 13% tapi IHSG nya turun 7 poin hmm apa hari ini GOTO akan terbang 🤣 https://t.co/K30s4cF7uN']

In [ ]:
df_goto_unicode = df_goto.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df_goto_unicode[df_goto_unicode.index == 6]['renderedContent'].to_list()

['Shm! Dapet saham gratis dari $GOTO bagusnya bikin sekuritas di mana ya?\\U0001f64f https://t.co/ZmooCcQteM',
 'PT GoTo Gojek Tokopedia Tbk mampu menghimpun dana hingga Rp 13,7 triliun dan memiliki kapitalisasi pasar mencapai Rp 400 triliun. #BEI m.jpnn.com/news/bei-goto-\\u2026',
 'Saham GOTO hari pertama naik 13% tapi IHSG nya turun 7 poin hmm apa hari ini GOTO akan terbang \\U0001f923 https://t.co/K30s4cF7uN']

In [ ]:
df_goto_decode = df_goto.applymap(lambda x: x.encode('utf-16', 'surrogatepass').decode('utf-16') if isinstance(x, str) else x)
df_goto_decode[df_goto_decode.index == 6]['renderedContent'].to_list()

['Shm! Dapet saham gratis dari $GOTO bagusnya bikin sekuritas di mana ya?🙏 https://t.co/ZmooCcQteM',
 'PT GoTo Gojek Tokopedia Tbk mampu menghimpun dana hingga Rp 13,7 triliun dan memiliki kapitalisasi pasar mencapai Rp 400 triliun. #BEI m.jpnn.com/news/bei-goto-…',
 'Saham GOTO hari pertama naik 13% tapi IHSG nya turun 7 poin hmm apa hari ini GOTO akan terbang 🤣 https://t.co/K30s4cF7uN']

In [ ]:
kode = 'GIAA'
nama = 'Garuda Indonesia'
since = '2010-10-29'
until = '2011-02-12'

df_tweets = get_tweets_union(kode, nama, since, until)
df_tweets['kode_saham'] = kode
df_tweets['nama_saham'] = nama
df_tweets['since_date'] = since
df_tweets['unitl_date'] = until


# df_tweets.to_excel(DESTFILE+kode+".xlsx")

In [ ]:
kode

'GIAA'

In [ ]:
df_tweets[df_tweets.index != 13192].to_excel(DESTFILE+kode+".xlsx")

In [ ]:
df_tweets[df_tweets.index == 13192].T.to_dict()

{13192: {'_type': 'snscrape.modules.twitter.Tweet',
  'url': 'https://twitter.com/adelavidia/status/19024865533100032',
  'date': '2010-12-26T13:40:56+00:00',
  'rawContent': 'AJUKAN JARI TENGAH KALIAN BUAT MALAYSIA !!!\nFUCK YOU MALAYSIA DARI KAMI GARUDA INDONESIA !!!\xad\x1c\xad',
  'renderedContent': 'AJUKAN JARI TENGAH KALIAN BUAT MALAYSIA !!!\nFUCK YOU MALAYSIA DARI KAMI GARUDA INDONESIA !!!\xad\x1c\xad',
  'id': '19024865533100032',
  'user': {'_type': 'snscrape.modules.twitter.User',
   'username': 'adelavidia',
   'id': 100431710,
   'displayname': 'Adela',
   'rawDescription': 'Be yourself■',
   'renderedDescription': 'Be yourself■',
   'descriptionLinks': None,
   'verified': False,
   'created': '2009-12-30T07:02:23+00:00',
   'followersCount': 1783,
   'friendsCount': 767,
   'statusesCount': 37057,
   'favouritesCount': 38,
   'listedCount': 3,
   'mediaCount': 130,
   'location': '',
   'protected': False,
   'link': {'_type': 'snscrape.modules.twitter.TextLink',
    'tex

In [ ]:
path_giaa = '/content/drive/MyDrive/Freelance/twitter_sentiment_analysis/data_new/output_old3/GIAA.xlsx'
df_giaa = pd.read_excel(path_giaa)

In [ ]:
df_giaa[df_giaa['id'] == 19024865533100032].T.to_dict()

{13204: {'Unnamed: 0': 13204,
  '_type': 'snscrape.modules.twitter.Tweet',
  'url': 'https://twitter.com/adelavidia/status/19024865533100032',
  'date': '2010-12-26T13:40:56+00:00',
  'rawContent': 'AJUKAN JARI TENGAH KALIAN BUAT MALAYSIA !!!\\nFUCK YOU MALAYSIA DARI KAMI GARUDA INDONESIA !!!\\xad\\x1c\\xad',
  'renderedContent': 'AJUKAN JARI TENGAH KALIAN BUAT MALAYSIA !!!\\nFUCK YOU MALAYSIA DARI KAMI GARUDA INDONESIA !!!\\xad\\x1c\\xad',
  'id': 19024865533100032,
  'user': "{'_type': 'snscrape.modules.twitter.User', 'username': 'adelavidia', 'id': 100431710, 'displayname': 'Adela', 'rawDescription': 'Be yourself■', 'renderedDescription': 'Be yourself■', 'descriptionLinks': None, 'verified': False, 'created': '2009-12-30T07:02:23+00:00', 'followersCount': 1783, 'friendsCount': 767, 'statusesCount': 37057, 'favouritesCount': 38, 'listedCount': 3, 'mediaCount': 130, 'location': '', 'protected': False, 'link': {'_type': 'snscrape.modules.twitter.TextLink', 'text': 'instagram.com//itsme

In [ ]:
len(df_giaa)

26168

## 4. Run Scrapping

In [ ]:
list_emitendone = os.listdir(DESTFILE)
list_emitendone = [li[:-5] for li in list_emitendone]
# list_emitendone
list_emiten = df_fileipo[['kode', 'nama', 'tanggal_ipo_plus_1', 'tgl_poin_1']]
list_emiten = list_emiten[~list_emiten['kode'].isin(list_emitendone)]
list_emiten.head()

,kode,nama,tanggal_ipo_plus_1,tgl_poin_1
0,EMDE,Megapolitan Developments,2011-01-13,2010-10-01
1,MBTO,Martina Berto,2011-01-14,2010-09-30
2,GIAA,PT Garuda Indonesia,2011-02-12,2010-10-29
3,MBSS,Mitrabahtera Segara Sejati,2011-04-07,2010-12-22
4,SRAJ,Sejahteraraya Anugrahjaya,2011-04-12,2010-12-27


In [ ]:
list_emiten = list_emiten.T.to_dict()
is_first = True
for emiten in list_emiten.values():
  kode = emiten['kode']
  nama = emiten['nama']
  since = emiten['tgl_poin_1'].strftime("%Y-%m-%d")
  until = emiten['tanggal_ipo_plus_1'].strftime("%Y-%m-%d")

  try:
    df_tweets = get_tweets_union(kode, nama, since, until)
    df_tweets['kode_saham'] = kode
    df_tweets['nama_saham'] = nama
    df_tweets['since_date'] = since
    df_tweets['unitl_date'] = until

    # if is_first:
    #   df_temp = df_tweets
    #   is_first = False
    # else:
    #   df_temp = pd.concat([df_temp, df_tweets])
    
    # print(kode, nama, since, until, len(df_tweets), len(df_temp))
    # df_tweets = df_tweets.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
    if kode == 'GIAA':
      df_tweets[df_tweets['id'] != 19024865533100032].to_excel(DESTFILE+kode+".xlsx")
    else:
      df_tweets.to_excel(DESTFILE+kode+".xlsx")
  except:
    print(kode)

In [ ]:
list_emitendone = df_fileipo['kode'].tolist()
is_first = True
for emitendone in list_emitendone:
  if is_first:
    df_all = pd.read_excel(DESTFILE+emitendone+".xlsx")
    is_first = False
  else:
    df_temp = pd.read_excel(DESTFILE+emitendone+".xlsx")
    df_all = pd.concat([df_all, df_temp])

df_all['id'] = df_all['id'].astype(str)
df_all['conversationId'] = df_all['conversationId'].astype(str)
df_all['inReplyToTweetId'] = df_all['inReplyToTweetId'].fillna(0).astype(np.int64).astype(str)
df_all.to_excel(DESTFILE+"all.xlsx")

In [7]:
df_all = pd.read_excel(DESTFILE+"all.xlsx", engine=None)
print(len(df_all))
df_all.head()
# 165783

165783


,Unnamed: 0.1,Unnamed: 0,_type,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,links,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe,kode_saham,nama_saham,since_date,unitl_date
0,0,0,snscrape.modules.twitter.Tweet,https://twitter.com/graha11/status/25121785301...,2011-01-12T09:27:55+00:00,Berita Properti: Megapolitan Developments List...,Berita Properti: Megapolitan Developments List...,25121785301180416,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,25121785301180416,in,"<a href=""http://twitterfeed.com"" rel=""nofollow...",http://twitterfeed.com,twitterfeed,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EMDE,Megapolitan Developments,2010-10-01,2011-01-13
1,1,1,snscrape.modules.twitter.Tweet,https://twitter.com/fixer_rudy_pt/status/25118...,2011-01-12T09:13:12+00:00,Kompas: Megapolitan Developments Listing di BE...,Kompas: Megapolitan Developments Listing di BE...,25118083190362112,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,25118083190362112,in,"<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",https://dlvrit.com/,dlvr.it,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EMDE,Megapolitan Developments,2010-10-01,2011-01-13
2,2,2,snscrape.modules.twitter.Tweet,https://twitter.com/achaidar/status/2511639805...,2011-01-12T09:06:30+00:00,Sunyalangu News .... Megapolitan Developments ...,Sunyalangu News .... Megapolitan Developments ...,25116398053236736,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,25116398053236736,in,"<a href=""https://www.google.com/"" rel=""nofollo...",https://www.google.com/,Google,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EMDE,Megapolitan Developments,2010-10-01,2011-01-13
3,3,3,snscrape.modules.twitter.Tweet,https://twitter.com/SM_Sekuritas/status/251141...,2011-01-12T08:57:40+00:00,"Mengawali 2011, PT Megapolitan Developments Tb...","Mengawali 2011, PT Megapolitan Developments Tb...",25114176036802560,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,25114176036802560,in,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://twitter.com,Twitter Web Client,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EMDE,Megapolitan Developments,2010-10-01,2011-01-13
4,4,4,snscrape.modules.twitter.Tweet,https://twitter.com/AiLapYuPul/status/25113862...,2011-01-12T08:56:26+00:00,Megapolitan Developments Listing di BEI http:/...,Megapolitan Developments Listing di BEI http:/...,25113862256730112,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,25113862256730112,in,"<a href=""http://twitterfeed.com"" rel=""nofollow...",http://twitterfeed.com,twitterfeed,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EMDE,Megapolitan Developments,2010-10-01,2011-01-13


## 5. Parse Nested Columns

In [8]:
user_keys = ['username',
            'id',
            'displayname',
            'rawDescription',
            'renderedDescription',
            'verified',
            'created',
            'followersCount',
            'friendsCount',
            'statusesCount',
            'favouritesCount',
            'listedCount',
            'mediaCount',
            'location',
            'protected',
            'profileImageUrl',
            'profileBannerUrl',
            'url']
for key in user_keys:
  df_all['user_'+key] = df_all['user'].apply(lambda x: ast.literal_eval(x)[key])

In [9]:
quotedTweets = ['date',
                'rawContent',
                'id',
                'lang']
for key in quotedTweets:
  df_all['quotedTweets_'+key] = df_all['quotedTweet'].apply(lambda x: ast.literal_eval(x)[key] if str(x) != 'nan' else 'nan')
df_all['quotedTweets_username'] = df_all['quotedTweet'].apply(lambda x: ast.literal_eval(x)['user']['username'] if str(x) != 'nan' else 'nan')

df_all = df_all.drop(columns=['user', 'quotedTweet'])

In [10]:
df_all = df_all.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', '_type', 'links', 'media', 'retweetedTweet', 'inReplyToUser', 'mentionedUsers', 
                              'coordinates', 'place', 'hashtags', 'cashtags', 'card', 'viewCount', 'vibe'])

## 6. Filter

In [11]:
df_allclean = df_all.drop_duplicates()
len(df_allclean)
#152571

152571

In [ ]:
df_allclean.head()

,url,date,rawContent,renderedContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,inReplyToTweetId,kode_saham,nama_saham,since_date,unitl_date,user_username,user_id,user_displayname,user_rawDescription,user_renderedDescription,user_verified,user_created,user_followersCount,user_friendsCount,user_statusesCount,user_favouritesCount,user_listedCount,user_mediaCount,user_location,user_protected,user_profileImageUrl,user_profileBannerUrl,user_url,quotedTweets_date,quotedTweets_rawContent,quotedTweets_id,quotedTweets_lang,quotedTweets_username
0,https://twitter.com/graha11/status/25121785301...,2011-01-12T09:27:55+00:00,Berita Properti: Megapolitan Developments List...,Berita Properti: Megapolitan Developments List...,25121785301180416,0,0,0,0,25121785301180416,in,"<a href=""http://twitterfeed.com"" rel=""nofollow...",http://twitterfeed.com,twitterfeed,0,EMDE,Megapolitan Developments,2010-10-01,2011-01-13,graha11,183261332,Graha Sebelas,http://t.co/op9zvGMWSD hadir untuk memberikan ...,Graha11.com hadir untuk memberikan fasilitas p...,False,2010-08-26T15:29:31+00:00,457,328,27158,0,2,1,Jakarta,False,https://pbs.twimg.com/profile_images/111229721...,None,https://twitter.com/graha11,nan,nan,nan,nan,nan
1,https://twitter.com/fixer_rudy_pt/status/25118...,2011-01-12T09:13:12+00:00,Kompas: Megapolitan Developments Listing di BE...,Kompas: Megapolitan Developments Listing di BE...,25118083190362112,0,0,0,0,25118083190362112,in,"<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",https://dlvrit.com/,dlvr.it,0,EMDE,Megapolitan Developments,2010-10-01,2011-01-13,fixer_rudy_pt,202626595,Rudy@PT,マーケッター兼トレーダー / EC、アフィリエイト、FX、etc... / 広告費0円で半年...,マーケッター兼トレーダー / EC、アフィリエイト、FX、etc... / 広告費0円で半年...,False,2010-10-14T13:30:38+00:00,5063,0,101865,0,48,8,"起業8ヶ月目で月収1,000万円稼いだマーケッターの全て⬇️",False,https://pbs.twimg.com/profile_images/116051397...,https://pbs.twimg.com/profile_banners/20262659...,https://twitter.com/fixer_rudy_pt,nan,nan,nan,nan,nan
2,https://twitter.com/achaidar/status/2511639805...,2011-01-12T09:06:30+00:00,Sunyalangu News .... Megapolitan Developments ...,Sunyalangu News .... Megapolitan Developments ...,25116398053236736,0,0,0,0,25116398053236736,in,"<a href=""https://www.google.com/"" rel=""nofollo...",https://www.google.com/,Google,0,EMDE,Megapolitan Developments,2010-10-01,2011-01-13,achaidar,227265364,Abu Chaidar,Seneng-seneng Aja,Seneng-seneng Aja,False,2010-12-16T11:03:01+00:00,50,9,215735,0,0,0,,False,https://pbs.twimg.com/profile_images/119242609...,None,https://twitter.com/achaidar,nan,nan,nan,nan,nan
3,https://twitter.com/SM_Sekuritas/status/251141...,2011-01-12T08:57:40+00:00,"Mengawali 2011, PT Megapolitan Developments Tb...","Mengawali 2011, PT Megapolitan Developments Tb...",25114176036802560,0,0,0,0,25114176036802560,in,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://twitter.com,Twitter Web Client,0,EMDE,Megapolitan Developments,2010-10-01,2011-01-13,SM_Sekuritas,228649152,Sinarmas Sekuritas,Official Twitter Sinarmas Sekuritas.\nDownload...,Official Twitter Sinarmas Sekuritas.\nDownload...,False,2010-12-20T09:03:25+00:00,4382,9,6959,39,50,1188,Plaza BII Tower III Lt.5,False,https://pbs.twimg.com/profile_images/968690257...,https://pbs.twimg.com/profile_banners/22864915...,https://twitter.com/SM_Sekuritas,nan,nan,nan,nan,nan
4,https://twitter.com/AiLapYuPul/status/25113862...,2011-01-12T08:56:26+00:00,Megapolitan Developments Listing di BEI http:/...,Megapolitan Developments Listing di BEI http:/...,25113862256730112,0,0,0,0,25113862256730112,in,"<a href=""http://twitterfeed.com"" rel=""nofollow...",http://twitterfeed.com,twitterfeed,0,EMDE,Megapolitan Developments,2010-10-01,2011-01-13,AiLapYuPul,62973657,Ai Lap Yu Pul,"Mbah Surip (born Urip Ariyanto, 6 May 1957 - 4...","Mbah Surip (born Urip Ariyanto, 6 May 1957 - 4...",False,2009-08-05T00:12:08+00:00,956,1576,310401,9,4,0,Tak Gendong Kemana-mana,False,https://pbs.twimg.com/profile_images/359890768...,N

In [12]:
df_allclean['until_date'] = pd.to_datetime(df_allclean['unitl_date'], errors='coerce')
len(df_allclean)

<ipython-input-12-ca1cc5963b05>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_allclean['until_date'] = pd.to_datetime(df_allclean['unitl_date'], errors='coerce')


152571

In [13]:
df_all_2021 = df_allclean[df_allclean['until_date'].dt.year != 2022]

len(df_all_2021)

128071

In [ ]:
df_all_2022q1 = df_allclean[df_allclean['until_date'].apply(lambda x: x.strftime("%Y%m")) <= '202203']

len(df_all_2022q1)

130106

In [ ]:
df_all_goto = df_allclean[df_allclean['until_date'].apply(lambda x: x.strftime("%Y%m%d")) <= '20220412']

len(df_all_goto)

133352

In [ ]:
df_all_2022q2 = df_allclean[df_allclean['until_date'].apply(lambda x: x.strftime("%Y%m")) <= '202206']

len(df_all_2022q2)

134245

## Cleansing & Combine with IPO Data

In [ ]:
# df_fileipo = pd.read_csv(FILENAME)
# df_tweets = pd.read_excel(DESTFILE+"gab_plus_offside.xlsx")
df_fileipo[df_fileipo.kode == 'GOTO']

,No.,Kode,Nama Emiten,Sektor,Tanggal Pencatatan IPO,Tanggal Penawaran Awal,Harga Penawaran Awal,Harga IPO,Masa Penawaran Umum,nama,kode,tanggal_penawaran_awal,tanggal_ipo,diff_days,28_days_date,tgl_awal_coalesced,tgl_poin_1,tanggal_ipo_plus_1
403,404,GOTO,PT GoTo Gojek Tokopedia Tbk,Technology,11-Apr-22,15 - 24 Mar 2022,316 - 346,338,1 - 7 April 2022,GoTo Gojek Tokopedia,GOTO,2022-03-15,2022-04-11,27.0,2022-03-14,2022-03-15,2021-12-28,2022-04-12


In [14]:
df_tojoin = df_all_2021
df_tojoin['kode_saham'] = df_tojoin['kode_saham'].apply(lambda x: 'TRUE' if x == True else x)

df_join = df_tojoin.merge(df_fileipo, left_on='kode_saham', right_on='kode')
df_join['rawContent'] = df_join['rawContent'].apply(lambda x: re.sub('[\r\n]', ' ', str(x)))
df_join['renderedContent'] = df_join['renderedContent'].apply(lambda x: re.sub('[\r\n]', ' ', str(x)))
df_join['user_rawDescription'] = df_join['user_rawDescription'].apply(lambda x: re.sub('[\r\n]', ' ', str(x)))
df_join['user_renderedDescription'] = df_join['user_renderedDescription'].apply(lambda x: re.sub('[\r\n]', ' ', str(x)))
df_join['quotedTweets_rawContent'] = df_join['quotedTweets_rawContent'].apply(lambda x: re.sub('[\r\n]', ' ', str(x)))
df_join.head()

<ipython-input-14-f8706e2c7217>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tojoin['kode_saham'] = df_tojoin['kode_saham'].apply(lambda x: 'TRUE' if x == True else x)


,url,date,rawContent,renderedContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,inReplyToTweetId,kode_saham,nama_saham,since_date,unitl_date,user_username,user_id,user_displayname,user_rawDescription,user_renderedDescription,user_verified,user_created,user_followersCount,user_friendsCount,user_statusesCount,user_favouritesCount,user_listedCount,user_mediaCount,user_location,user_protected,user_profileImageUrl,user_profileBannerUrl,user_url,quotedTweets_date,quotedTweets_rawContent,quotedTweets_id,quotedTweets_lang,quotedTweets_username,until_date,No.,Kode,Nama Emiten,Sektor,Tanggal Pencatatan IPO,Tanggal Penawaran Awal,Harga Penawaran Awal,Harga IPO,Masa Penawaran Umum,nama,kode,tanggal_penawaran_awal,tanggal_ipo,diff_days,28_days_date,tgl_awal_coalesced,tgl_poin_1,tanggal_ipo_plus_1,tanggal_penawaran_umum,pblcoffr,tanggal_penawaran_umum_coalesce
0,https://twitter.com/graha11/status/25121785301...,2011-01-12T09:27:55+00:00,Berita Properti: Megapolitan Developments List...,Berita Properti: Megapolitan Developments List...,25121785301180416,0,0,0,0,25121785301180416,in,"<a href=""http://twitterfeed.com"" rel=""nofollow...",http://twitterfeed.com,twitterfeed,0,EMDE,Megapolitan Developments,2010-10-01,2011-01-13,graha11,183261332,Graha Sebelas,http://t.co/op9zvGMWSD hadir untuk memberikan ...,Graha11.com hadir untuk memberikan fasilitas p...,False,2010-08-26T15:29:31+00:00,457,328,27158,0,2,1,Jakarta,False,https://pbs.twimg.com/profile_images/111229721...,None,https://twitter.com/graha11,nan,nan,nan,nan,nan,2011-01-13,1,EMDE,Megapolitan Developments Tbk.,Properties & Real Estate,12-Jan-11,17 - 22 Dec 2010,150 - 250,250,4 - 6 Jan 2011,Megapolitan Developments,EMDE,2010-12-17,2011-01-12,26.0,2010-12-15,2010-12-17,2010-10-01,2011-01-13,2011-01-04,2011-01-04,2011-01-04
1,https://twitter.com/fixer_rudy_pt/status/25118...,2011-01-12T09:13:12+00:00,Kompas: Megapolitan Developments Listing di BE...,Kompas: Megapolitan Developments Listing di BE...,25118083190362112,0,0,0,0,25118083190362112,in,"<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",https://dlvrit.com/,dlvr.it,0,EMDE,Megapolitan Developments,2010-10-01,2011-01-13,fixer_rudy_pt,202626595,Rudy@PT,マーケッター兼トレーダー / EC、アフィリエイト、FX、etc... / 広告費0円で半年...,マーケッター兼トレーダー / EC、アフィリエイト、FX、etc... / 広告費0円で半年...,False,2010-10-14T13:30:38+00:00,5063,0,101865,0,48,8,"起業8ヶ月目で月収1,000万円稼いだマーケッターの全て⬇️",False,https://pbs.twimg.com/profile_images/116051397...,https://pbs.twimg.com/profile_banners/20262659...,https://twitter.com/fixer_rudy_pt,nan,nan,nan,nan,nan,2011-01-13,1,EMDE,Megapolitan Developments Tbk.,Properties & Real Estate,12-Jan-11,17 - 22 Dec 2010,150 - 250,250,4 - 6 Jan 2011,Megapolitan Developments,EMDE,2010-12-17,2011-01-12,26.0,2010-12-15,2010-12-17,2010-10-01,2011-01-13,2011-01-04,2011-01-04,2011-01-04
2,https://twitter.com/achaidar/status/2511639805...,2011-01-12T09:06:30+00:00,Sunyalangu News .... Megapolitan Developments ...,Sunyalangu News .... Megapolitan Developments ...,25116398053236736,0,0,0,0,25116398053236736,in,"<a href=""https://www.google.com/"" rel=""nofollo...",https://www.google.com/,Google,0,EMDE,Megapolitan Developments,2010-10-01,2011-01-13,achaidar,227265364,Abu Chaidar,Seneng-seneng Aja,Seneng-seneng Aja,False,2010-12-16T11:03:01+00:00,50,9,215735,0,0,0,,False,https://pbs.twimg.com/profile_images/119242609...,None,https://twitter.com/achaidar,nan,nan,nan,nan,nan,2011-01-13,1,EMDE,Megapolitan Developments Tbk.,Properties & Real Estate,12-Jan-11,17 - 22 Dec 2010,150 - 250,250,4 - 6 Jan 2011,Megapolitan Developments,EMDE,2010-12-17,2011-01-12,26.0,2010-12-15,2010-12-17,2010-10-01,2011-01-13,2011-01-04,2011-01-04,2011-01-04
3,https://twitter.com/SM_Sekuritas/status/251141...,2011-01-12T08:57:40+00:00,"Mengawali 2011, PT Megapolitan Developments Tb...","Mengawali 2011, PT Megapolitan Developments Tb...",25114176036802560,0,0,0,0,25114176036802560,in,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://tw

In [15]:
df_join['date'] = df_join['date'].apply(parser.parse)
df_join['tanggal_penawaran_umum_coalesce'] = pd.to_datetime(df_join['tanggal_penawaran_umum_coalesce'])
df_join['date_differ'] = (df_join['tanggal_penawaran_umum_coalesce'].dt.date - df_join['date'].dt.date) / np.timedelta64(1, 'D')
df_join['fase'] = df_join['date_differ'].apply(lambda x: 'fase 1' if x > 0 else 'fase 2')
df_join.head()

,url,date,rawContent,renderedContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,inReplyToTweetId,kode_saham,nama_saham,since_date,unitl_date,user_username,user_id,user_displayname,user_rawDescription,user_renderedDescription,user_verified,user_created,user_followersCount,user_friendsCount,user_statusesCount,user_favouritesCount,user_listedCount,user_mediaCount,user_location,user_protected,user_profileImageUrl,user_profileBannerUrl,user_url,quotedTweets_date,quotedTweets_rawContent,quotedTweets_id,quotedTweets_lang,quotedTweets_username,until_date,No.,Kode,Nama Emiten,Sektor,Tanggal Pencatatan IPO,Tanggal Penawaran Awal,Harga Penawaran Awal,Harga IPO,Masa Penawaran Umum,nama,kode,tanggal_penawaran_awal,tanggal_ipo,diff_days,28_days_date,tgl_awal_coalesced,tgl_poin_1,tanggal_ipo_plus_1,tanggal_penawaran_umum,pblcoffr,tanggal_penawaran_umum_coalesce,date_differ,fase
0,https://twitter.com/graha11/status/25121785301...,2011-01-12 09:27:55+00:00,Berita Properti: Megapolitan Developments List...,Berita Properti: Megapolitan Developments List...,25121785301180416,0,0,0,0,25121785301180416,in,"<a href=""http://twitterfeed.com"" rel=""nofollow...",http://twitterfeed.com,twitterfeed,0,EMDE,Megapolitan Developments,2010-10-01,2011-01-13,graha11,183261332,Graha Sebelas,http://t.co/op9zvGMWSD hadir untuk memberikan ...,Graha11.com hadir untuk memberikan fasilitas p...,False,2010-08-26T15:29:31+00:00,457,328,27158,0,2,1,Jakarta,False,https://pbs.twimg.com/profile_images/111229721...,None,https://twitter.com/graha11,nan,nan,nan,nan,nan,2011-01-13,1,EMDE,Megapolitan Developments Tbk.,Properties & Real Estate,12-Jan-11,17 - 22 Dec 2010,150 - 250,250,4 - 6 Jan 2011,Megapolitan Developments,EMDE,2010-12-17,2011-01-12,26.0,2010-12-15,2010-12-17,2010-10-01,2011-01-13,2011-01-04,2011-01-04,2011-01-04,-8.0,fase 2
1,https://twitter.com/fixer_rudy_pt/status/25118...,2011-01-12 09:13:12+00:00,Kompas: Megapolitan Developments Listing di BE...,Kompas: Megapolitan Developments Listing di BE...,25118083190362112,0,0,0,0,25118083190362112,in,"<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",https://dlvrit.com/,dlvr.it,0,EMDE,Megapolitan Developments,2010-10-01,2011-01-13,fixer_rudy_pt,202626595,Rudy@PT,マーケッター兼トレーダー / EC、アフィリエイト、FX、etc... / 広告費0円で半年...,マーケッター兼トレーダー / EC、アフィリエイト、FX、etc... / 広告費0円で半年...,False,2010-10-14T13:30:38+00:00,5063,0,101865,0,48,8,"起業8ヶ月目で月収1,000万円稼いだマーケッターの全て⬇️",False,https://pbs.twimg.com/profile_images/116051397...,https://pbs.twimg.com/profile_banners/20262659...,https://twitter.com/fixer_rudy_pt,nan,nan,nan,nan,nan,2011-01-13,1,EMDE,Megapolitan Developments Tbk.,Properties & Real Estate,12-Jan-11,17 - 22 Dec 2010,150 - 250,250,4 - 6 Jan 2011,Megapolitan Developments,EMDE,2010-12-17,2011-01-12,26.0,2010-12-15,2010-12-17,2010-10-01,2011-01-13,2011-01-04,2011-01-04,2011-01-04,-8.0,fase 2
2,https://twitter.com/achaidar/status/2511639805...,2011-01-12 09:06:30+00:00,Sunyalangu News .... Megapolitan Developments ...,Sunyalangu News .... Megapolitan Developments ...,25116398053236736,0,0,0,0,25116398053236736,in,"<a href=""https://www.google.com/"" rel=""nofollo...",https://www.google.com/,Google,0,EMDE,Megapolitan Developments,2010-10-01,2011-01-13,achaidar,227265364,Abu Chaidar,Seneng-seneng Aja,Seneng-seneng Aja,False,2010-12-16T11:03:01+00:00,50,9,215735,0,0,0,,False,https://pbs.twimg.com/profile_images/119242609...,None,https://twitter.com/achaidar,nan,nan,nan,nan,nan,2011-01-13,1,EMDE,Megapolitan Developments Tbk.,Properties & Real Estate,12-Jan-11,17 - 22 Dec 2010,150 - 250,250,4 - 6 Jan 2011,Megapolitan Developments,EMDE,2010-12-17,2011-01-12,26.0,2010-12-15,2010-12-17,2010-10-01,2011-01-13,2011-01-04,2011-01-04,2011-01-04,-8.0,fase 2
3,https://twitter.com/SM_Sekuritas/status/251141...,2011-01-12 08:57:40+00:00,"Mengawali 2011, PT Megapolitan Developments Tb...","Mengawali 2011, PT Megapolitan Developments Tb...",25114176036802560,0,0,0,0,25114176036802560,in,"<a href=""h

In [ ]:
df_join.groupby('kode').count()['id'].sort_values(ascending=False).head()

kode
PPRO    5243
WSKT    5186
GLOB    4259
BIRD    4085
SMBR    3187
Name: id, dtype: int64

In [ ]:
df_join.groupby('fase').count()['id']

fase
fase 1    61460
fase 2    66611
Name: id, dtype: int64

In [19]:
exclude_kode = ('CPGT', 'NAGA', 'DAJK', 'FINN', 'AMOR', 'SKBM', 'BRIS')
df_filter = df_join[~df_join['kode'].isin(exclude_kode)]
df_filter.to_csv(DESTFILE+"data_all_raw_excld_7emiten.csv")
len(df_filter)

125126

In [25]:
# df_join = pd.read_csv(DESTFILE+"data_all_raw_excld_7emiten.csv")
# df_join['date'] = df_join['date'].apply(parser.parse)
# df_join['tgl_awal_coalesced'] = pd.to_datetime(df_join['tgl_awal_coalesced'], format='%Y-%m-%d')
# df_join['date_differ'] = (df_join['tgl_awal_coalesced'].dt.date - df_join['date'].dt.date) / np.timedelta64(1, 'D')
# df_join['fase'] = df_join['date_differ'].apply(lambda x: 'fase 1' if x >= 0 else 'fase 2')
# df_join.tail()

In [51]:
df_join = pd.read_csv(DESTFILE+"data_all_raw_excld7emiten.csv")
df_join['date'] = df_join['date'].apply(parser.parse)
df_join['date_differ'] = (pd.to_datetime(df_join['tanggal_penawaran_umum_coalesce']).dt.date - df_join['date'].dt.date) / np.timedelta64(1, 'D')
df_join['date_differ_preipo'] = (pd.to_datetime(df_join['tgl_poin_1']).dt.date - df_join['date'].dt.date) / np.timedelta64(1, 'D')
df_join['fase'] = df_join['date_differ'].apply(lambda x: 'fase 1' if x > 0 else 'fase 2')
df_join['is_in_preipofase'] = df_join['date_differ_preipo'].apply(lambda x: 1 if x > 0 else 0)
df_join.head(1)

<ipython-input-51-69468616a019>:1: DtypeWarning: Columns (38,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df_join = pd.read_csv(DESTFILE+"data_all_raw_excld7emiten.csv")


,Unnamed: 0,url,date,rawContent,renderedContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,inReplyToTweetId,kode_saham,nama_saham,since_date,unitl_date,user_username,user_id,user_displayname,user_rawDescription,user_renderedDescription,user_verified,user_created,user_followersCount,user_friendsCount,user_statusesCount,user_favouritesCount,user_listedCount,user_mediaCount,user_location,user_protected,user_profileImageUrl,user_profileBannerUrl,user_url,quotedTweets_date,quotedTweets_rawContent,quotedTweets_id,quotedTweets_lang,quotedTweets_username,until_date,No.,Kode,Nama Emiten,Sektor,Tanggal Pencatatan IPO,Tanggal Penawaran Awal,Harga Penawaran Awal,Harga IPO,Masa Penawaran Umum,nama,kode,tanggal_penawaran_awal,tanggal_ipo,diff_days,28_days_date,tgl_awal_coalesced,tgl_poin_1,tanggal_ipo_plus_1,tanggal_penawaran_umum,pblcoffr,tanggal_penawaran_umum_coalesce,date_differ,fase,date_differ_preipo,is_in_preipofase
0,0,https://twitter.com/graha11/status/25121785301...,2011-01-12 09:27:55+00:00,Berita Properti: Megapolitan Developments List...,Berita Properti: Megapolitan Developments List...,25121785301180416,0,0,0,0,25121785301180416,in,"<a href=""http://twitterfeed.com"" rel=""nofollow...",http://twitterfeed.com,twitterfeed,0,EMDE,Megapolitan Developments,2010-10-01,2011-01-13,graha11,183261332,Graha Sebelas,http://t.co/op9zvGMWSD hadir untuk memberikan ...,Graha11.com hadir untuk memberikan fasilitas p...,False,2010-08-26T15:29:31+00:00,457,328,27158,0,2,1,Jakarta,False,https://pbs.twimg.com/profile_images/111229721...,NaN,https://twitter.com/graha11,NaN,NaN,NaN,NaN,NaN,2011-01-13,1,EMDE,Megapolitan Developments Tbk.,Properties & Real Estate,12-Jan-11,17 - 22 Dec 2010,150 - 250,250.0,4 - 6 Jan 2011,Megapolitan Developments,EMDE,2010-12-17,2011-01-12,26.0,2010-12-15,2010-12-17,2010-10-01,2011-01-13,2011-01-04,2011-01-04,2011-01-04,-8.0,fase 2,-103.0,0


In [53]:
df_test = df_join.groupby(['kode_saham', 'fase']).count().reset_index()
fase_2 = df_test[df_test['fase'] == 'fase 2'][['kode_saham', 'url']].set_index('kode_saham')
fase_1 = df_test[df_test['fase'] == 'fase 1'][['kode_saham', 'url']].set_index('kode_saham')

fase_2.merge(fase_1, left_on='kode_saham', right_on='kode_saham', how='left').count()

url_x    382
url_y    358
dtype: int64

In [56]:
df_test = df_join.groupby(['fase']).count()['date'].reset_index()
df_test

,fase,date
0,fase 1,59707
1,fase 2,65419


In [57]:
FILES = lambda x : "df_text_only_excld7emiten_{}.csv".format(x)
for i in range(2011, 2023):
  df_temp = df_join[pd.to_datetime(df_join['tanggal_ipo']).dt.year == i]
  df_temp = df_temp[['renderedContent', 'kode', 'fase', 'tanggal_ipo', 'date']]
  print(i, FILES(i), len(df_temp))
  df_temp.to_csv(DESTFILE+FILES(i))

2011 df_text_only_excld7emiten_2011.csv 9059
2012 df_text_only_excld7emiten_2012.csv 22903
2013 df_text_only_excld7emiten_2013.csv 19806
2014 df_text_only_excld7emiten_2014.csv 26422
2015 df_text_only_excld7emiten_2015.csv 19489
2016 df_text_only_excld7emiten_2016.csv 7098
2017 df_text_only_excld7emiten_2017.csv 3851
2018 df_text_only_excld7emiten_2018.csv 4177
2019 df_text_only_excld7emiten_2019.csv 3778
2020 df_text_only_excld7emiten_2020.csv 4205
2021 df_text_only_excld7emiten_2021.csv 4338
2022 df_text_only_excld7emiten_2022.csv 0


In [59]:
df_text_only = df_join[['renderedContent', 'kode', 'fase', 'tanggal_ipo', 'date']]
df_text_only.to_csv(DESTFILE+"data_text_only_excld7emiten.csv")